# Geodéziai koordináta rendszer transzformációk

In [3]:
!pip install pyproj

import numpy as np
from pyproj import CRS, Transformer, Proj
from geopy.distance import geodesic

## Bevezetés


Egy pontot a térben természetes módon megadhatunk X, Y, Z descartes-i koordinátákkal. A geodéziában a térbeli koordináta rendszert gyakran a Föld középpontjához és forgásához kötnek, ekkor geocentrikus koordináta rendszerről beszélünk, vagy külföldi szakirodalomban gyakran ([ECEF](https://en.wikipedia.org/wiki/Earth-centered,_Earth-fixed_coordinate_system)) koordináta rendszerként hivatkoznak rá. 

A térbeli koordináta rendszerben adott pont koordinátái viszonylag nagy számok, mely számok alapján nehéz megmondani a pont körübelüli helyzetét a Földön. Ezért a három X, Y, Z koordináta helyett, a gyakorlatban gyakran alkalmazzunk szög alapú (hosszúság és szélesség) koordináta definíciót; ezek megadásához azonban gömb vagy ellipszoid használata szükséges.

A geodéziai alapfelület, vegy geodéziai dátum, az ellipszoid méretére és alakjára vonatkozó adatok, kiegészítve az ellipszoid elhelyezésével és esetlegesen a tájékozásával kapcsolatos paraméterekkel. Az ellipszoid elhelyezése lehetséges úgy, hogy az a világ minden pontján a legjobban közelítse a geoidot (pl. WGS84, ETRS89, NAD83), vagy pedig úgy is, hogy egy adott területet jobban közelít; ekkor lokális dátumról beszélünk (pl. ED50, NAD27)

Egy másik nyomós érv a geodézi dátum, és így a szög alapú koordináta definiálás mellett, hogy geodéziában a vizszintes és magassági koordináták gyakran külön kezeltek azok gyakorlati különbözőségei miatt. Hasonló módon, térinformatikai rendszerkben a vizszintes és magassági dátum esetenként külön kezeltek. Ekkor a koordináta transzformációkor a transzformációt végző programok a magasság értékeket nem kerül adják meg, vagy a bemenettel megegyező érték kerül visszaadásra.

Ezek alapján egy pont megadása a következő módokon történik az EPSG definíciókban:

- **Térbeli koordináta**: három X, Y, Z descartes-i koordánátával adható meg egy pont. Példák [EPSG:4978](https://epsg.io/4936) (WGS84), [EPSG:4936](https://epsg.io/4936) (ETRS89)
- **Ellipszoidi koordináta**: egy térbeli pont megadásához a hosszúság, szélesség és esetlegesen ellipszoid vagy geoid feletti magasságot használjuk. Ahhoz hogy a hosszúság és szélességet értelmezni tudjuk, egy ellipszoidot illesztenek egy adott térbeli koordináta rendszerhez. Példák [EPSG:4326](https://epsg.io/4326) (WGS84), [EPSG:4937](https://epsg.io/4937) (ETRS89)
- **Vetületi koordináta**: 

Irodalom:

- [Koordináta rendszerek](https://pygis.io/docs/d_crs_what_is_it.html)

## EOV

In [2]:
crs_eov = CRS.from_epsg(23700) # HD72 / EOV
crs_etrs89_llh = CRS.from_epsg(4258) # ETRS89/GRS80, lat/lon
crs_etrs89_xyz = CRS.from_epsg(4936) # ETRS89, ECEF-XYZ

print('EOV/HD72: ')
print(crs_eov.to_wkt(pretty=True))

EOV/HD72: 
PROJCRS["HD72 / EOV",
    BASEGEOGCRS["HD72",
        DATUM["Hungarian Datum 1972",
            ELLIPSOID["GRS 1967",6378160,298.247167427,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4237]],
    CONVERSION["Egyseges Orszagos Vetuleti",
        METHOD["Hotine Oblique Mercator (variant B)",
            ID["EPSG",9815]],
        PARAMETER["Latitude of projection centre",47.1443937222222,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8811]],
        PARAMETER["Longitude of projection centre",19.0485717777778,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8812]],
        PARAMETER["Azimuth of initial line",90,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8813]],
        PARAMETER["Angle from Rectified to Skew Grid",90,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8814]],

In [4]:
# %%
eov2etrs89_xyz = Transformer.from_crs(crs_eov, crs_etrs89_xyz)
eov2etrs89_llh = Transformer.from_crs(crs_eov, crs_etrs89_llh)

In [5]:
# %%
# BUTE GPS Point: https://www.gnssnet.hu/pdf/BUTE.pdf
y_eov = 650684.479 # easting
x_eov = 237444.175 # northing
z_eov = 137.177 # EOMA height

In [10]:
p_etrs89_xyz = eov2etrs89_xyz.transform(y_eov, x_eov, z_eov)
p_etrs89_llh = eov2etrs89_llh.transform(y_eov, x_eov, z_eov)

In [11]:
# %% Comparison to eht2: http://eht.gnssnet.hu/index.php
p_eht_etrs89_xyz = np.array([4081882.378, 1410011.142, 4678199.391])
p_eht_etrs89_llh = np.array([47.4809437284, 19.0565297521, 180.811])

print(p_etrs89_xyz - p_eht_etrs89_xyz)

[-27.79323961  -9.71344038 -32.2041526 ]


In [12]:
# %% Geodetic distance between two points defined with lat/lon
coords_1 = (52.2296756, 21.0122287)
coords_2 = (52.406374, 16.9251681)
sol_1 = geodesic(coords_1, coords_2, ellipsoid='GRS-80').m * 100
sol_2 = geodesic(coords_1, coords_2, ellipsoid='WGS-84').m * 100
print('Difference GRS-80 vs. WGS-84 [cm]: ', sol_1 - sol_2)

Difference GRS-80 vs. WGS-84 [cm]:  0.00028607621788978577


In [13]:
# %%
print('Difference XY [cm]: ', geodesic(p_eht_etrs89_llh[0:2], p_etrs89_llh[0:2]).m * 100)
# Note: the Z of p_etrs89_llh[2] is the same as input, no Z coordinate transformation
# print('Difference Z  [m]: ', (p_eht_etrs89_llh[2] - p_etrs89_llh[2]))


Difference XY [cm]:  12.471731150364025


In [14]:
# %% Original definiation of HD72/EOV in proj4 format
print('Original definiation:')
print(crs_eov.to_proj4())

Original definiation:
+proj=somerc +lat_0=47.1443937222222 +lon_0=19.0485717777778 +k_0=0.99993 +x_0=650000 +y_0=200000 +ellps=GRS67 +units=m +no_defs +type=crs


/home/zoltan/env/uni/lib/python3.8/site-packages/pyproj/crs/crs.py:1286: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)


In [34]:
# %%
# Grid letölthető: https://github.com/OSGeoLabBp/eov2etrs
# < transzformáció 1 cm-es középhibán belül azonos az EHT-vel
# az EHT transzformációja 10 cm-es középhibájú
nadgrids = '/home/zoltan/Repo/uni/geoprog-lecture-notes/geoprog-lecture-notes/data/etrs2eov_notowgs.gsb'# XY grid
geoidgrids = '/home/zoltan/Repo/uni/geoprog-lecture-notes/geoprog-lecture-notes/data/geoid_eht2014.gtx' # geoid model for Z transformation
crs_eov_grid = Proj(init='EPSG:23700', nadgrids=nadgrids, geoidgrids=geoidgrids) # HD72 / EOV
print(crs_eov_grid.to_proj4())

+proj=somerc +lat_0=47.1443937222222 +lon_0=19.0485717777778 +k_0=0.99993 +x_0=650000 +y_0=200000 +ellps=GRS67 +nadgrids=/home/zoltan/Repo/uni/geoprog-lecture-notes/geoprog-lecture-notes/data/etrs2eov_notowgs.gsb +units=m +geoidgrids=/home/zoltan/Repo/uni/geoprog-lecture-notes/geoprog-lecture-notes/data/geoid_eht2014.gtx +geoid_crs=WGS84 +vunits=m +no_defs


/home/zoltan/env/uni/lib/python3.8/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [37]:
# %% XYZ transformation
eov2etrs89_xyz = Transformer.from_proj(crs_eov_grid, crs_etrs89_xyz)
p_etrs89_xyz = eov2etrs89_xyz.transform(y_eov, x_eov, z_eov)
print(p_etrs89_xyz)
print(p_eht_etrs89_xyz)
print('Difference [cm]: ', np.linalg.norm(p_etrs89_xyz - p_eht_etrs89_xyz) * 100)

(4081882.3770687627, 1410011.1433535905, 4678199.3908524215)
[4081882.378 1410011.142 4678199.391]
Difference [cm]:  0.16496028796703496


In [36]:
# %% LLH transformation:
eov2etrs89_llh = Transformer.from_proj(crs_eov_grid, crs_etrs89_llh)
p_etrs89_llh = eov2etrs89_llh.transform(y_eov, x_eov, z_eov)
print('Difference [cm]: ', geodesic(p_etrs89_llh[0:2], p_eht_etrs89_llh[0:2]).m * 100)
print('Difference Z  [cm]: ', (p_etrs89_llh[2] - p_eht_etrs89_llh[2])*100)

Difference [cm]:  0.16408611991976702
Difference Z  [cm]:  -4363.4000000000015


In [28]:
print(p_eht_etrs89_llh)

[ 47.48094373  19.05652975 180.811     ]
